In [1]:
import numpy as np
import pandas as pd
import orangecontrib.associate.fpgrowth as oaf
import datetime
import argparse
#parser = argparse.ArgumentParser()
#parser.add_argument('--train_dir', type=str, default='./trainDataHJ.txt')
#parser.add_argument('--rules_dir', type=str, default='./rulesDataHJ.txt')
#parser.add_argument('--support', type=float, default=0.05)
#parser.add_argument('--confidence',type=float,default=0.2)
#args = parser.parse_args()

In [2]:
y=np.random.randint(1,6,[1000,])
hum=np.random.randint(0,101,[1000,])
tem=np.random.uniform(-50.0,40.0,[1000,])
is_gas=np.random.randint(0,2,1000)
sur_tem=np.random.uniform(-50.0,100.0,[1000,])
is_dew=np.random.randint(0,2,1000)
mach_cat=np.random.randint(1,4,[1000,])
meth_cat=np.random.randint(1,4,[1000,])


In [3]:
df=pd.DataFrame({
    "hum":hum,
    "tem":tem,
    "is_gas":is_gas,
    "sur_tem":sur_tem,
    "is_dew":is_dew,
    "mach_cat":mach_cat,
    "meth_cat":meth_cat,
    "time":y
})

In [5]:
#df.to_csv('trainDataHJ.csv',index=False)

In [6]:
def data_pre(path):
    df=pd.read_csv(path)
    #iscretization
    bins=[0,50,70,90,100]
    df['hum']=pd.cut(df['hum'],bins,right=False,labels=['1','2','3','4'])
    bins=[-50.0,-20.0,-10.0,0,10.0,20.0,30.0,41.0]
    df['tem']=pd.cut(df['tem'],bins,right=False,labels=['1','2','3','4','5','6','7'])
    bins=[-50.0,-20.0,-10.0,0,10.0,20.0,30.0,60.0,100.0]
    df['sur_tem']=pd.cut(df['sur_tem'],bins,right=False,labels=['1','2','3','4','5','6','7','8'])
    #type trans
    df['is_gas']=df['is_gas'].apply(str)
    df['is_dew']=df['is_dew'].apply(str)
    df['mach_cat']=df['mach_cat'].apply(str)
    df['meth_cat']=df['meth_cat'].apply(str)
    df['time']=df['time'].apply(str)
    #one_hot
    df=pd.get_dummies(df)
    strDecode=df.columns.values.tolist()
    data=df.values
    data.dtype=bool
    return data,strDecode

In [7]:
def model(data,support=0.05,confidence=0.2):
    fre_ite=dict(oaf.frequent_itemsets(data,support))  #这里设置置信度
    rules = oaf.association_rules(fre_ite,confidence)
    result=list(rules)
    return result

In [8]:
def dealResult(rules,strDecode):
    returnRules = []
    for i in rules:
        j= list(i[1])
        if len(j)==1 and strDecode[j[0]].startswith('time') :
            temStr = [];
            temp=''
            for k in i[0]:
                temp = temp+strDecode[k]+'&'
            temp = temp[:-1]
            temp = temp + ' ==> '
            temp = temp +strDecode[j[0]] 
            temStr.append(temp)
            temStr.append(i[2])
            temStr.append(i[3])
            returnRules.append(temStr)
    returnRules=list(sorted(returnRules,key=lambda x:(-x[2],-x[1])))
    return returnRules
    


In [9]:
support=0.05
confidence=0.2
train_dir='trainDataHJ.csv'
rules_dir='rulesDataHJ.txt'
data,strDecode=data_pre(train_dir)
result=model(data,support,confidence)
printRules= dealResult(result,strDecode)
file=open(rules_dir,'w')
print('Rules'+'\t'+'support'+'\t'+'confidence'+'\n')
file.write('Rules'+'\t'+'support'+'\t'+'confidence'+'\n')
for line in printRules:
    s='\t'.join([str(i) for i in line])
    print(s+'\n')
    file.write(s+'\n')
file.close()
    

Rules	support	confidence

hum_3 ==> time_5	55	0.27638190954773867

tem_1 ==> time_5	84	0.2709677419354839

sur_tem_8 ==> time_2	66	0.24812030075187969

is_gas_1&is_dew_1 ==> time_2	57	0.24050632911392406

is_gas_0&is_dew_1 ==> time_5	59	0.23790322580645162

mach_cat_1 ==> time_5	81	0.23684210526315788

hum_1&is_dew_1 ==> time_1	52	0.23529411764705882

is_gas_0 ==> time_5	121	0.23495145631067962

meth_cat_2 ==> time_5	68	0.23367697594501718

meth_cat_3 ==> time_1	82	0.23295454545454544

is_gas_0&is_dew_0 ==> time_5	62	0.23220973782771537

hum_1&is_dew_0 ==> time_3	61	0.22761194029850745

is_gas_1&is_dew_1 ==> time_1	53	0.22362869198312235

is_gas_1 ==> time_2	108	0.22268041237113403

is_dew_1 ==> time_2	108	0.22268041237113403

mach_cat_3 ==> time_1	73	0.22121212121212122

sur_tem_8 ==> time_1	58	0.21804511278195488

is_gas_1&is_dew_0 ==> time_4	54	0.21774193548387097

hum_1&is_gas_1 ==> time_1	54	0.21686746987951808

hum_1 ==> time_1	106	0.2167689161554192

hum_1&is_gas_0 ==> time_1	52